# Advent of Code 2024 Day 16 

### Setup

In [79]:
from aocd import get_data, submit

day = 16
year = 2024


In [80]:
with open('example.txt', 'r') as file:
    raw_sample_data = "".join(file.readlines())

raw_sample_data[:100]

'#################\n#...#...#...#..E#\n#.#.#.#.#.#.#.#.#\n#.#.#.#...#...#.#\n#.#.#.#.###.#.#.#\n#...#.#.#.'

In [81]:
raw_test_data = get_data(day=day, year=year)

raw_test_data[:]

'#############################################################################################################################################\n#.....#...#.......#.............#...#...........#...........#.#.......#.............#...#.......#.........#...#...#........................E#\n#.#.#.#.#.#.#####.#.#.###.#####.###.#.###.#####.#.#.#######.#.#.#####.#########.###.#.#.#.#######.#.#####.#.###.#.#.###.#####.#.###########.#\n#.#.#.#.#...#.....#.#...#.....#...#...#...#.....#.#.#...#...#.#.........................#.......#.#.....#.#.....#.#.#...#.....#.......#...#.#\n###.#.#.#.#######.#####.#.#.#####.#.###.#######.#.#.#.#.#.###.###.#####.#.###.###.#.#.#.#.#####.#.#####.#.#######.#.#.###.###########.#.#.###\n#...#.#.#.#.....#.#...#...#...#...#.#...#.........#...#.#.#...#.#.................#...#.#.....#.#.#...#.#.....#...#.#.....#.....#.#...#.#...#\n#.###.#.#.#.###.#.#.#.#.#.#.#.#.###.#.###.#####.#.#####.#.#.#.#.###.#######.###########.###.#.#.#.###.#.#####.#.#.#.#######.#.#.#.#.###.###.#

##### Data Parsing

In [82]:
import numpy as np

def parse_data(raw_data:str):
    arr = np.array([list(line) for line in raw_data.splitlines()])
    return {
        'array': arr,
        'start': np.argwhere(arr == 'S').flatten(),
        'end': np.argwhere(arr == 'E').flatten(),
    }

     
sample_data = parse_data(raw_sample_data)
test_data = parse_data(raw_test_data)

sample_data['start'], sample_data['end']

(array([15,  1]), array([ 1, 15]))

### Part One!

In [83]:
use_sample_data = False
part = 'a'

In [84]:
data = sample_data if use_sample_data else test_data


In [85]:
from typing import Literal, Union, List, Set, Dict, Tuple

Wall = Literal['#']
Path = Literal['.']
Start = Literal['S']
End = Literal['E']

Tile = Union[Wall, Path, Start, End]

North = Literal['^']
East = Literal['>']
South = Literal['v']
West = Literal['<']

Direction = Union[North, East, South, West]

In [86]:
from functools import total_ordering

@total_ordering
class KeyDict(object):
    def __init__(self, key, dct):
        self.key = key
        self.dct = dct

    def __lt__(self, other):
        return self.key < other.key

    def __eq__(self, other):
        return self.key == other.key

    def __repr__(self):
        return '{0.__class__.__name__}(key={0.key}, dct={0.dct})'.format(self)
    
    def __iter__(self):
        return iter((self.key, self.dct))
    
    def __getitem__(self, key):
        return self.dct[key]
    
    def __setitem__(self, key, value):
        self.dct[key] = value

In [87]:
def get_next_pos(pos:np.ndarray, direction:Direction) -> np.ndarray:
    if direction == '^':
        return pos + np.array([-1, 0])
    if direction == '>':
        return pos + np.array([0, 1])
    if direction == 'v':
        return pos + np.array([1, 0])
    if direction == '<':
        return pos + np.array([0, -1])

In [88]:
def euclidean_distance(a, b):
    return np.sqrt(np.sum((a - b) ** 2))

def manhattan_distance(a, b):
    return np.sum(np.abs(a - b))

def dummy_heuristic(a, b):
    return 0

In [89]:
def cost(a, b, a_direction:Direction, b_direction:Direction):
    return 1 if a_direction == b_direction else 1000

In [90]:
import heapq

def astar(array:np.ndarray, start:np.ndarray, end:np.ndarray, default_direction: Direction = '>', heuristic=manhattan_distance, cost=cost):
    queue = [(0, {'pos': start, 'path': [], 'turns': [], 'direction': default_direction })]
    seen = set()

    while queue:
        score, current = heapq.heappop(queue)
        pos = current['pos']
        dir = current['direction']
        path = current['path']
        turns = current['turns']

        if pos[0] == end[0] and pos[1] == end[1]:
            return {
                'start': start,
                'end': end,
                'path': current['path'],
                'turns': current['turns'],
                'score': score,
            }
        
        seen.add((tuple(pos), dir))

        # moving forward 
        next_pos = get_next_pos(pos, dir)
        if array[next_pos[0], next_pos[1]] != '#' and (tuple(next_pos), dir) not in seen:
            next_pos_score = score + heuristic(next_pos, end) + cost(pos, next_pos, dir, dir)
            heapq.heappush(queue, KeyDict(next_pos_score, {
                'pos': next_pos,
                'path': path + [next_pos],
                'turns': turns,
                'direction': dir,
            }))
            

        # all turns 
        for next_dir in ['^', '>', 'v', '<']:
            if next_dir == dir or (tuple(pos), next_dir) in seen:
                continue

            next_pos = pos
            next_pos_score = score + heuristic(next_pos, end) + cost(pos, next_pos, dir, next_dir)
            obj = {
                'pos': next_pos,
                'path': path,
                'turns': turns + [(pos, next_dir)],
                'direction': next_dir,
            }
            
            heapq.heappush(queue, KeyDict(next_pos_score, obj))
    
    return None

In [91]:
def plot_path(array:np.ndarray, path:List[np.ndarray], turns:List[np.ndarray], starting_direction='>', start_pos:np.ndarray=None):
    arr = array.copy()
    direction = starting_direction
    turns_dict = {tuple(turn[0]): turn[1] for turn in turns}

    if start_pos is not None:
        path = [start_pos] + path

    for pos in path:
        if tuple(pos) in turns_dict:
            direction = turns_dict[tuple(pos)]

        if arr[pos[0], pos[1]] == '.':
            arr[pos[0], pos[1]] = direction

    np.printoptions(threshold=np.inf, linewidth=np.inf)
    print(arr)

In [92]:
search_result = astar(**data, heuristic=dummy_heuristic)
plot_path(data['array'], search_result['path'], search_result['turns'], '>', data['start'])

part_a_answer = search_result['score']
part_a_answer

[['#' '#' '#' ... '#' '#' '#']
 ['#' '.' '.' ... '>' 'E' '#']
 ['#' '.' '#' ... '#' '.' '#']
 ...
 ['#' '.' '#' ... '#' '.' '#']
 ['#' 'S' '>' ... '.' '.' '#']
 ['#' '#' '#' ... '#' '#' '#']]


108504

In [93]:
if not use_sample_data and part == 'a':
    submit(answer=part_a_answer, part='a', day=day, year=year, reopen=True)

aocd will not submit that answer again. At 2024-12-17 21:21:59.030907-05:00 you've previously submitted 108504 and the server responded with:
That's the right answer!  You are one gold star closer to finding the Chief Historian. [Continue to Part Two]


### Part Two!

In [94]:
use_sample_data = False
part='b'

In [95]:
data = sample_data if use_sample_data else test_data

In [96]:
import heapq

def astar_get_best_paths(array:np.ndarray, start:np.ndarray, end:np.ndarray, default_direction: Direction = '>', heuristic=manhattan_distance, cost=cost):
    queue = [(0, {'pos': start, 'path': [], 'turns': [], 'direction': default_direction })]
    seen = set()
    best_scores = [] 
    while queue:
        score, current = heapq.heappop(queue)
        pos = current['pos']
        dir = current['direction']
        path = current['path']
        turns = current['turns']

        if pos[0] == end[0] and pos[1] == end[1]:
            heapq.heappush(best_scores, KeyDict(score, {
                'start': start,
                'end': end,
                'path': current['path'],
                'turns': current['turns'],
                'score': score,
            }))
        
        if best_scores and best_scores[0].key < score:
            break

        seen.add((tuple(pos), dir))

        # moving forward 
        next_pos = get_next_pos(pos, dir)
        if array[next_pos[0], next_pos[1]] != '#' and (tuple(next_pos), dir) not in seen:
            next_pos_score = score + heuristic(next_pos, end) + cost(pos, next_pos, dir, dir)
            heapq.heappush(queue, KeyDict(next_pos_score, {
                'pos': next_pos,
                'path': path + [next_pos],
                'turns': turns,
                'direction': dir,
            }))
            

        # all turns 
        for next_dir in ['^', '>', 'v', '<']:
            if next_dir == dir or (tuple(pos), next_dir) in seen:
                continue

            next_pos = pos
            next_pos_score = score + heuristic(next_pos, end) + cost(pos, next_pos, dir, next_dir)
            obj = {
                'pos': next_pos,
                'path': path,
                'turns': turns + [(pos, next_dir)],
                'direction': next_dir,
            }
            
            heapq.heappush(queue, KeyDict(next_pos_score, obj))
    
    return best_scores

In [97]:
search_result = astar_get_best_paths(**data, heuristic=dummy_heuristic)

uniuqe_tiles = set([tuple(data['start'])])
for k, v in search_result:
    for pos in v['path']:
        uniuqe_tiles.add(tuple(pos))

part_b_answer = len(uniuqe_tiles)
part_b_answer

64

In [98]:
if not use_sample_data and part == 'b':
    submit(answer=part_b_answer, part='b', day=day, year=year, reopen=True)